In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from core.text_detection import DB_text_detection
from core.text_detection import EAST_text_detection
from core.segmentation import char_segmentation
from config.config_helpers import Config
from core.segmentation import word_segmentation

from utils.image_utils.Image import showImage
from utils.image_utils.Image import Image_preprocessing
from utils.image_utils.Image import image_padding
from utils.image_utils.Image import simple_threshold
from utils.image_utils import warp_image_crop

image_path = f"datasets\\sample_data\\tx.png"
json_config = Config(f"config\\config.json")
DB_model_path = f"models\\{json_config.get('image.DB_detection_model')}"
EAST_model_path = f"models\\{json_config.get('image.EAST_detection_model')}"

image = cv.imread(image_path)
frame = Image_preprocessing(image, json_config.get('image.scaling_factor'))
DB_frame = DB_text_detection(frame._thresh, DB_model_path)
EAST_frame = EAST_text_detection(frame._thresh, EAST_model_path)

output_size = json_config.get('image.output_image_size')

In [3]:
# showImage(frame._thresh)
# print(json_config.get('image.EAST_detection_model'))

# showImage(DB_frame._bound_image)

In [5]:
boxes, confidences = warp_image_crop.sort_boxes(DB_frame._boxes, DB_frame._confidence)
croppedLineImage = warp_image_crop.crop_image(frame._default_cvImage, boxes)
# showImage(croppedLineImage[0])

# word = word_segmentation(croppedLineImage[1])
# showImage(word._cropped_image[3])

char = char_segmentation(word._cropped_image[0])

showImage(char._bound_image)

NameError: name 'word' is not defined

In [16]:
# # data loader function definition
# predict_list = []
# char_list = []
# al = 0
# # baris ke kata
# for lines in croppedLineImage:
#     wordImage = word_segmentation(lines)
#     # print(f"ada {len(wordImage._cropped_image)} kata")

#     word_list = []

#     # error handling for empty char detection
#     if wordImage._cropped_image:
#         for word in wordImage._cropped_image:
#             charImage = char_segmentation(word)

#             padding_list = []

#             if charImage._mask:
#                 for mask in charImage._mask:
#                     mask = np.clip(mask, 0, 255).astype(np.uint8)
#                     padded_image = image_padding(mask, image_size=(output_size, output_size), interpolation=0)
#                     padding_list.append(padded_image)

#                     # charlist space app
#                     char_list.append('char')

#             word_list.append(padding_list)

#             # charlist space
#             char_list.append('space')

#     predict_list.append(word_list)
#     char_list.append('break')

# display_image = predict_list[0][3][2]
# plt.imshow(display_image, cmap='gray')
# print(display_image.shape)
# plt.show()

In [17]:
# data loader function definition
char_list = []
for lines in croppedLineImage:
    wordImage = word_segmentation(lines)

    # error handling for empty char detection
    if wordImage._cropped_image:
        for index, word in enumerate(wordImage._cropped_image):
            charImage = char_segmentation(word)

            if charImage._mask:
                for mask in charImage._mask:
                    mask = np.clip(mask, 0, 255).astype(np.uint8)
                    padded_image = image_padding(mask, image_size=(output_size, output_size), interpolation=0)

                    # charlist
                    char_list.append(['char', padded_image])

            # charlist
            if index == len(wordImage._cropped_image) - 1:
                pass
            else:
                char_list.append(['space', ' '])

    char_list.append(['break', '\n'])

In [18]:
from core.char import char_batch_predict
import time
import tensorflow as tf
from tensorflow.keras.models import load_model
from tqdm import tqdm
import re

# json_config.get('models.ResNet')[0] for model ResNet50
# json_config.get('models.ResNet')[1] for model ResNet101
char_recognition_model_path = f"models\\ResNet\\{json_config.get('models.ResNet')[0]}"
class_names = json_config.get('models.label')

char = char_batch_predict(char_recognition_model_path, class_names, char_list)

[info] loading model from model_path: models\ResNet\Chars74K_ResNet50.keras
[info] creating batch dataset
[info] batch dataset created
[info] len of batches is 3, with size per batch: 32


processing batches:  33%|███▎      | 1/3 [00:04<00:08,  4.02s/it]

Batch 1/3 processed


processing batches:  67%|██████▋   | 2/3 [00:04<00:02,  2.13s/it]

Batch 2/3 processed


processing batches: 100%|██████████| 3/3 [00:07<00:00,  2.62s/it]

Batch 3/3 processed
[info] result time for batch prediction: 7.88 seconds


In [19]:
print(char._text_result)

It waS Che beSt Of
timcS1 it waS Chc wOrSt
Of cimeS1 it waS che agc
Of wiSdOm1 it waS che
agc Of fOO1iShncSS

